In [2]:
pip install requests backoff

Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
import json
import backoff
import random

In [60]:
# https://docs.awesomeapi.com.br/
url = "https://economia.awesomeapi.com.br/json/last/USD-BRL"
ret = requests.get(url)

In [61]:
# if ret -> identifies automatically if the return is good.
if ret:
    print(ret)
else:
    print('request error')

<Response [200]>


In [62]:
json.loads(ret.text)

{'USDBRL': {'code': 'USD',
  'codein': 'BRL',
  'name': 'Dólar Americano/Real Brasileiro',
  'high': '5.2712',
  'low': '5.2705',
  'varBid': '0.0008',
  'pctChange': '0.02',
  'bid': '5.2703',
  'ask': '5.2713',
  'timestamp': '1666231086',
  'create_date': '2022-10-19 22:58:06'}}

In [63]:
usd = json.loads(ret.text)['USDBRL']
print(f"20 dollars costs today: {float(usd['bid']) * 20} reais")

20 dólars costs today: 105.40599999999999 reais


In [50]:
def quotation(value, currency):
    url = f"https://economia.awesomeapi.com.br/json/last/{currency}"
    ret = requests.get(url)
    currency_price = json.loads(ret.text)[currency.replace('-','')]
    print(f"{value} {currency:3} costs today: {float(currency_price['bid']) * 20} {currency[-3:]}")

In [51]:
quotation(20, 'USD-BRL')

20 USD-BRL costs today: 105.40599999999999 BRL


In [52]:
quotation(20, 'JPY-BRL')

20 JPY-BRL costs today: 0.7030000000000001 BRL


In [53]:
try:
    quotation(20, 'USD-BRL')
except Exception as e:
    # pass
    print(e)
else:
    print("request ok")

20 USD-BRL costs today: 105.40599999999999 BRL
request ok


In [64]:
currency_list = [
    "USD-BRL",
    "EUR-BRL",
    "BTC-BRL",
    "RPL-BRL", # inexistent
    "JPY-BRL",
]

for currency in currency_list:
    try:
        quotation(20, currency)
    except:
        #pass
        print(f"{currency} failed")

20 USD-BRL costs today: 105.40599999999999 BRL
20 EUR-BRL costs today: 102.834 BRL
20 BTC-BRL costs today: 2022.46 BRL
RPL-BRL failed
20 JPY-BRL costs today: 0.7030000000000001 BRL


In [67]:
def error_check(func): #a decorator
    def inner_func(*args, **kargs):
        try:
            func(*args, **kargs)
        except:
            print(f"{func.__name__} has failed.")
    return inner_func

In [68]:
@error_check
def quotation(value, currency):
    url = f"https://economia.awesomeapi.com.br/json/last/{currency}"
    ret = requests.get(url)
    currency_price = json.loads(ret.text)[currency.replace('-','')]
    print(f"{value} {currency:3} costs today: {float(currency_price['bid']) * 20} {currency[-3:]}")

In [69]:
currency_list = [
    "USD-BRL",
    "EUR-BRL",
    "BTC-BRL",
    "RPL-BRL", # inexistent
    "JPY-BRL",
]

for currency in currency_list:
    try:
        quotation(20, currency)
    except:
        #pass
        print(f"{currency} failed")

20 USD-BRL costs today: 105.40599999999999 BRL
20 EUR-BRL costs today: 102.834 BRL
20 BTC-BRL costs today: 2022.46 BRL
quotation has failed.
20 JPY-BRL costs today: 0.7030000000000001 BRL


In [13]:
# Example to explain args and kwargs in Python functions + using backoff package to handle error and retries
# max_time and max_tries
@backoff.on_exception(backoff.expo, (ConnectionAbortedError, ConnectionRefusedError, TimeoutError), max_tries=2)
def test_func(*args, **kargs):
    random_n = random.random()
    print(f"""
            RND: {random_n}
            args: {args if args else 'no args'}
            kwargs: {kargs if kargs else 'no kwargs'}
    """)
    if random_n < .2:
        raise ConnectionAbortedError('Connection was finalized')
    elif random_n < .4:
        raise ConnectionRefusedError('Connection was denied')
    elif random_n < .6:
        raise TimeoutError('Awaiting time exceed')
    else:
        return "OK!"

In [14]:
test_func()


            RND: 0.21243098571648422
            args: no args
            kwargs: no kwargs
    

            RND: 0.0924009027149656
            args: no args
            kwargs: no kwargs
    


ConnectionAbortedError: Connection was finalized

In [15]:
test_func(52)


            RND: 0.5591897577071525
            args: (52,)
            kwargs: no kwargs
    

            RND: 0.8884258621068949
            args: (52,)
            kwargs: no kwargs
    


'OK!'

In [16]:
test_func(42, 51, name='will')


            RND: 0.7866223049630375
            args: (42, 51)
            kwargs: {'name': 'will'}
    


'OK!'